In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import cv2 as cv

In [19]:
data_iris = load_iris()
df_iris = pd.DataFrame(data_iris.data, columns=data_iris.feature_names)
df_iris['target'] = data_iris.target
df_iris['target_names'] = df_iris['target'].apply(lambda x: data_iris.target_names[x])
#df_iris

In [20]:
#count the number of each target
df_iris['target_names'].value_counts()

setosa        50
versicolor    50
virginica     50
Name: target_names, dtype: int64

In [21]:
features = [col for col in df_iris.columns if col not in ["target", "target_names"]]
df_iris_features = df_iris[features]

In [22]:
training_features, testing_features, training_labels, testing_labels = train_test_split(df_iris_features, df_iris['target'], test_size=0.3, random_state=42)

In [23]:
# find mean and standard deviation for each feature
training_features.groupby(training_labels).agg(['mean', 'std'])

sepal length (cm)       sepal width (cm)       petal length (cm)        \
                    mean   std             mean   std              mean   std   
target                                                                          
0                  4.965 0.340            3.377 0.376             1.465 0.185   
1                  5.862 0.532            2.724 0.299             4.211 0.496   
2                  6.559 0.659            2.986 0.315             5.546 0.544   

       petal width (cm)        
                   mean   std  
target                         
0                 0.248 0.109  
1                 1.303 0.206  
2                 2.005 0.297

In [24]:
#fine mean and standard deviation for each feature
mean_setosa, mean_versicolor, mean_virginica = training_features.groupby(training_labels).mean().values
std_setosa, std_versicolor, std_virginica = training_features.groupby(training_labels).std().values

In [25]:
def gaussian(x, mean, std):
    return 1/(np.sqrt(2*np.pi)*std)*np.exp(-((x-mean)**2)/(2*std**2))
    
def predict(sepal_length, sepal_width, petal_length, petal_width):
    return (1/3)*sepal_length*sepal_width*petal_length*petal_width

In [26]:
#setosa probability
p_setosa_sepal_length, p_setosa_sepal_width = gaussian(testing_features['sepal length (cm)'], mean_setosa[0], std_setosa[0]), gaussian(testing_features['sepal width (cm)'], mean_setosa[1], std_setosa[1])
p_setosa_petal_length, p_setosa_petal_width = gaussian(testing_features['petal length (cm)'], mean_setosa[2], std_setosa[2]), gaussian(testing_features['petal width (cm)'], mean_setosa[3], std_setosa[3])
p_setosa = predict(p_setosa_sepal_length, p_setosa_sepal_width, p_setosa_petal_length, p_setosa_petal_width)

#versicolor probability
p_versicolor_sepal_length, p_versicolor_sepal_width = gaussian(testing_features['sepal length (cm)'], mean_versicolor[0], std_versicolor[0]), gaussian(testing_features['sepal width (cm)'], mean_versicolor[1], std_versicolor[1])
p_versicolor_petal_length, p_versicolor_petal_width = gaussian(testing_features['petal length (cm)'], mean_versicolor[2], std_versicolor[2]), gaussian(testing_features['petal width (cm)'], mean_versicolor[3], std_versicolor[3])
p_versicolor = predict(p_versicolor_sepal_length, p_versicolor_sepal_width, p_versicolor_petal_length, p_versicolor_petal_width)

#virginica probability
p_virginica_sepal_length, p_virginica_sepal_width = gaussian(testing_features['sepal length (cm)'], mean_virginica[0], std_virginica[0]), gaussian(testing_features['sepal width (cm)'], mean_virginica[1], std_virginica[1])
p_virginica_petal_length, p_virginica_petal_width = gaussian(testing_features['petal length (cm)'], mean_virginica[2], std_virginica[2]), gaussian(testing_features['petal width (cm)'], mean_virginica[3], std_virginica[3])
p_virginica = predict(p_virginica_sepal_length, p_virginica_sepal_width, p_virginica_petal_length, p_virginica_petal_width)

In [27]:
df_ans = pd.DataFrame()
df_ans['p_setosa'] = p_setosa
df_ans['p_versicolor'] = p_versicolor
df_ans['p_virginica'] = p_virginica
df_ans['target_names'] = testing_labels.apply(lambda x: data_iris.target_names[x])
# what column has the highest probability
df_ans['my_predicted_labels'] = df_ans[['p_setosa', 'p_versicolor', 'p_virginica']].idxmax(axis=1)
df_ans['my_predicted_labels'] = df_ans['my_predicted_labels'].apply(lambda x: x.split('_')[1])
df_ans['Correct_Naive'] = df_ans['target_names'] == df_ans['my_predicted_labels']
#df_ans

In [28]:
# t0tal accuracy
from sklearn.metrics import accuracy_score
accuracy_percent = accuracy_score(df_ans['target_names'], df_ans['my_predicted_labels']) * 100
print ("Accuracy ['NB from Scratch']: %5.2f%%" %accuracy_percent)

Accuracy ['NB from Scratch']: 97.78%


In [29]:
# use Guussian Naive Bayes from sklearn
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
model = classifier.fit(training_features, training_labels)
predictions_labels = model.predict(testing_features)

# create a dataframe to compare the results
df_ans['GaussianNB'] = predictions_labels
df_ans['GaussianNB'] = df_ans['GaussianNB'].apply(lambda x: data_iris.target_names[x])
df_ans['correct_GaussianNB'] = df_ans['target_names'] == df_ans['GaussianNB']
accuracy_percent = accuracy_score(testing_labels, predictions_labels) * 100
print ("Accuracy ['sklearn.naive_bayes']: %5.2f%%" %accuracy_percent)
df_ans['check equal'] = df_ans['Correct_Naive'] == df_ans['correct_GaussianNB']

Accuracy ['sklearn.naive_bayes']: 97.78%


In [30]:

if all(df_ans['check equal']):
    print('success')
else:
    print('fail')

success


In [31]:
# import matplotlib.pyplot as plt
# import pandas as pd

#  # the value have 3 decimal places
# pd.options.display.float_format = '{:,.3f}'.format
# fig, ax = plt.subplots(figsize=(20, 2))
# ax.xaxis.set_visible(False)  # hide the x axis
# ax.yaxis.set_visible(False)  # hide the y axis

# pd.plotting.table(ax, df_ans)  # where df is your data frame

# plt.savefig('mytable.png')